# Import thư viện

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, mean, stddev, lit
import numpy

In [3]:
spark = SparkSession.builder.appName("Customer_Segmentation").getOrCreate()

# Load data

In [29]:
df_train = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/content/Train.csv")
df_test = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/content/Test.csv")
df_train = df_train.drop("Segmentation")
df_train = df_train.drop("Var_1")
df_test = df_test.drop("Var_1")

In [30]:
# Gom hai DataFrame lại với nhau
df = df_train.union(df_test)

Hàm "explain()" dùng để hiển thị kế hoạch thực thi các phép biến đổi và thực hiện trên DataFrame.

Nó sẽ cung cấp các thông tin:


*   Định dạng dữ liệu được đọc (.csv).
*   Chỉ số ID của các cột (cột "ID" có ID = 17; cột "Gender" -> 18; ....; cột "Segmentation" -> 27).
*   Batched = False => dữ liệu đọc theo từng dòng thay vì đọc một khối dữ liệu lớn.
*   DataFilters: [] => rỗng cho thấy ko có áp dụng **bộ lọc dữ liệu** nào được áp dụng.
*   Location: đường dẫn là "/content/Train"
*   PartitionFilters: [] => rỗng cho thế không có **bộ lọc phân vùng** nào được áp dụng.
*   PushedFilters: [] => rỗng cho thấy không có **bộ lọc quét tệp** nào được áp dụng.
*   ReadSchema: Schema của dữ liệu bao gồm tên cột và kiểu dữ liệu của mỗi cột

In [31]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Ever_Married: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Graduated: string (nullable = true)
 |-- Profession: string (nullable = true)
 |-- Work_Experience: double (nullable = true)
 |-- Spending_Score: string (nullable = true)
 |-- Family_Size: double (nullable = true)



In [32]:
df.explain()

== Physical Plan ==
Union
:- FileScan csv [ID#706,Gender#707,Ever_Married#708,Age#709,Graduated#710,Profession#711,Work_Experience#712,Spending_Score#713,Family_Size#714] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/Train.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<ID:int,Gender:string,Ever_Married:string,Age:int,Graduated:string,Profession:string,Work_E...
+- FileScan csv [ID#745,Gender#746,Ever_Married#747,Age#748,Graduated#749,Profession#750,Work_Experience#751,Spending_Score#752,Family_Size#753] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/Test.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<ID:int,Gender:string,Ever_Married:string,Age:int,Graduated:string,Profession:string,Work_E...




In [33]:
df.show()

+------+------+------------+---+---------+-------------+---------------+--------------+-----------+
|    ID|Gender|Ever_Married|Age|Graduated|   Profession|Work_Experience|Spending_Score|Family_Size|
+------+------+------------+---+---------+-------------+---------------+--------------+-----------+
|462809|  Male|          No| 22|       No|   Healthcare|            1.0|           Low|        4.0|
|462643|Female|         Yes| 38|      Yes|     Engineer|           NULL|       Average|        3.0|
|466315|Female|         Yes| 67|      Yes|     Engineer|            1.0|           Low|        1.0|
|461735|  Male|         Yes| 67|      Yes|       Lawyer|            0.0|          High|        2.0|
|462669|Female|         Yes| 40|      Yes|Entertainment|           NULL|          High|        6.0|
|461319|  Male|         Yes| 56|       No|       Artist|            0.0|       Average|        2.0|
|460156|  Male|          No| 32|      Yes|   Healthcare|            1.0|           Low|        3.0|


In [34]:
print("DataFrame ", (df.count(), len(df.columns)))

DataFrame  (10695, 9)


In [35]:
# Tạo một danh sách chứa các tên cột và số lượng giá trị bị thiếu tương ứng
missing_values = []

# Lặp qua các cột trong DataFrame
for col in df.columns:
    # Tính tổng số giá trị bị thiếu trong cột và thêm vào danh sách
    missing_count = df.filter(df[col].isNull()).count()
    missing_values.append((col, missing_count))
# Hiển thị kết quả
print("Thống kê giá trị Null trong tập DataFrame:\n" + "-"*20)
for col, missing_count in missing_values:
    if missing_count != 0:
        print(f" - '{col}': {missing_count} giá trị Null.")


Thống kê giá trị Null trong tập DataFrame:
--------------------
 - 'Ever_Married': 190 giá trị Null.
 - 'Graduated': 102 giá trị Null.
 - 'Profession': 162 giá trị Null.
 - 'Work_Experience': 1098 giá trị Null.
 - 'Family_Size': 448 giá trị Null.


Nhận xét: Có tổng cộng 5 cột bị thiếu dữ liệu trong cả tập train và tập test đều là: Ever_Married, Graduated, Profession, Work_Experience, Family_Size.

# Một số Hàm

## Hàm tính mode() trong cột

In [36]:
# Hàm tính giá trị xuất hiện nhiều nhất trong cột
def calculate_mode(df, column_name):
    mode_df = (df.groupBy(column_name)  # gom nhóm lại theo cột
               .count()                 # đếm số lần xuất hiện của mỗi giá trị trong cột
               .orderBy("count", ascending=False)   # sắp xếp theo thứ tự giảm dần về tần suất xuất hiện
               .first())               # Lấy phần tử đầu tiên, tức nó là giá trị xuất hiện nhiều nhất trong cột
    return mode_df[column_name]

## Hàm tính mean() trong cột

In [37]:
# Hàm tính giá trị trung bình trong cột
def calculate_mean(df, column_name):
    mean_value = df.select(mean(column_name)).collect()[0][0]
    return mean_value

## Hàm chuẩn hóa StandardScaler

In [38]:
def standard_scaler(df, columns):
    # Tính toán mean và standard deviation từ df
    mean_values = {col_name: df.select(mean(col_name)).collect()[0][0] for col_name in columns}
    std_dev_values = {col_name: df.select(stddev(col_name)).collect()[0][0] for col_name in columns}

    # Chuẩn hóa df
    for col_name in columns:
        mean_val = mean_values[col_name]
        std_dev_val = std_dev_values[col_name]
        df = df.withColumn(col_name, (df[col_name] - lit(mean_val)) / lit(std_dev_val))

    return df

## One hot encoder

## Hàm mã hóa nhị phân các biến phân loại thành số

In [39]:
def encoder(dataframe, columns):
    for col in columns:
        indexer = StringIndexer(inputCol=col, outputCol=f"{col}_encoded", handleInvalid="keep")
        dataframe = indexer.fit(dataframe).transform(dataframe)
        dataframe = dataframe.drop(col)
        dataframe = dataframe.withColumnRenamed(f"{col}_encoded", col)
    return dataframe

Hàm này sẽ giữ lại các giá trị Null chứ không gán Null thành 1 con số nào đó.

# Tiền xử lý

In [40]:
df.show()

+------+------+------------+---+---------+-------------+---------------+--------------+-----------+
|    ID|Gender|Ever_Married|Age|Graduated|   Profession|Work_Experience|Spending_Score|Family_Size|
+------+------+------------+---+---------+-------------+---------------+--------------+-----------+
|462809|  Male|          No| 22|       No|   Healthcare|            1.0|           Low|        4.0|
|462643|Female|         Yes| 38|      Yes|     Engineer|           NULL|       Average|        3.0|
|466315|Female|         Yes| 67|      Yes|     Engineer|            1.0|           Low|        1.0|
|461735|  Male|         Yes| 67|      Yes|       Lawyer|            0.0|          High|        2.0|
|462669|Female|         Yes| 40|      Yes|Entertainment|           NULL|          High|        6.0|
|461319|  Male|         Yes| 56|       No|       Artist|            0.0|       Average|        2.0|
|460156|  Male|          No| 32|      Yes|   Healthcare|            1.0|           Low|        3.0|


## Xóa dữ liệu trùng lặp

In [41]:
print("DataFrame ", (df.count(), len(df.columns)))

DataFrame  (10695, 9)


In [42]:
df = df.dropDuplicates() # dùng để xóa các mẫu dữ liệu trùng lập với nhau trong tập train

In [43]:
print("DataFrame ", (df.count(), len(df.columns)))

DataFrame  (10559, 9)


## Mã hóa các biến phân loại thành số

In [44]:
# Tính giá trị xuất hiện nhiều nhất trong cột "Profession" của tập train
numeric_columns = ["Profession"]
mode_value_train = calculate_mode(df, 'Profession')

# Điền các giá trị bị thiếu trong cột "Profession" của tập train bằng giá trị xuất hiện nhiều nhất
df = df.na.fill({column: mode_value_train for column in numeric_columns})

In [45]:
# Định nghĩa các cột phân loại cho cả tập train và tập test
categorical_columns = ["Profession", "Spending_Score"]

In [47]:
profession_mapping = {
    "Artist": 1,
    "Healthcare": 2,
    "Entertainment": 3,
    "Engineer": 4,
    "Doctor": 5,
    "Lawyer": 6,
    "Executive": 7,
    "Marketing": 8,
    "Homemaker": 9
}

df = df.withColumn(
    "Profession",
    when(df["Profession"] == "Artist", profession_mapping['Artist'])
    .when(df["Profession"] == "Healthcare", profession_mapping['Healthcare'])
    .when(df["Profession"] == "Entertainment", profession_mapping['Entertainment'])
    .when(df["Profession"] == "Engineer", profession_mapping['Engineer'])
    .when(df["Profession"] == "Doctor", profession_mapping['Doctor'])
    .when(df["Profession"] == "Lawyer", profession_mapping['Lawyer'])
    .when(df["Profession"] == "Executive", profession_mapping['Executive'])
    .when(df["Profession"] == "Marketing", profession_mapping['Marketing'])
    .when(df["Profession"] == "Homemaker", profession_mapping['Homemaker'])
    .otherwise(None)
)

In [48]:
"""# Tạo danh sách các bước của Pipeline cho từng cột phân loại trong categorical_columns
stages = []
for col in categorical_columns:
    # Bước 1: Sử dụng StringIndexer để chuyển đổi các giá trị phân loại thành các chỉ số số
    string_indexer = StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="skip")
    # Bước 2: Sử dụng OneHotEncoder để mã hóa các chỉ số số thành vector nhị phân
    one_hot_encoder = OneHotEncoder(inputCol=col + "_index", outputCol=col + "_encoded")
    stages += [string_indexer, one_hot_encoder]

# Xây dựng Pipeline
pipeline = Pipeline(stages=stages)

# Áp dụng Pipeline cho DataFrame
pipeline_model = pipeline.fit(df)
# Transform cả tập train và tập test bằng pipeline đã fit
df = pipeline_model.transform(df)

# Xóa các cột phân loại gốc và các cột chỉ số số tạm thời cho categorical_columns
for col in categorical_columns:
    df = df.drop(col).drop(col + "_index")

# Đổi tên các cột đã mã hóa về tên gốc cho categorical_columns
for col in categorical_columns:
    df = df.withColumnRenamed(col + "_encoded", col)"""

'# Tạo danh sách các bước của Pipeline cho từng cột phân loại trong categorical_columns\nstages = []\nfor col in categorical_columns:\n    # Bước 1: Sử dụng StringIndexer để chuyển đổi các giá trị phân loại thành các chỉ số số\n    string_indexer = StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="skip")\n    # Bước 2: Sử dụng OneHotEncoder để mã hóa các chỉ số số thành vector nhị phân\n    one_hot_encoder = OneHotEncoder(inputCol=col + "_index", outputCol=col + "_encoded")\n    stages += [string_indexer, one_hot_encoder]\n\n# Xây dựng Pipeline\npipeline = Pipeline(stages=stages)\n\n# Áp dụng Pipeline cho DataFrame\npipeline_model = pipeline.fit(df)\n# Transform cả tập train và tập test bằng pipeline đã fit\ndf = pipeline_model.transform(df)\n\n# Xóa các cột phân loại gốc và các cột chỉ số số tạm thời cho categorical_columns\nfor col in categorical_columns:\n    df = df.drop(col).drop(col + "_index")\n\n# Đổi tên các cột đã mã hóa về tên gốc cho categorical_columns\n

In [49]:
df.show()

+------+------+------------+---+---------+----------+---------------+--------------+-----------+
|    ID|Gender|Ever_Married|Age|Graduated|Profession|Work_Experience|Spending_Score|Family_Size|
+------+------+------------+---+---------+----------+---------------+--------------+-----------+
|462809|  Male|          No| 22|       No|         2|            1.0|             0|        4.0|
|460666|Female|         Yes| 49|      Yes|         9|           14.0|             0|        1.0|
|463622|  Male|          No| 27|      Yes|         2|            8.0|             0|        1.0|
|467826|  Male|          No| 18|       No|         2|            0.0|             0|        6.0|
|462040|  Male|          No| 19|       No|         2|            5.0|             0|        4.0|
|467926|  Male|         Yes| 79|      Yes|         1|            0.0|             2|        2.0|
|467761|  Male|          No| 39|      Yes|         1|            1.0|             0|        2.0|
|459076|  Male|         Yes| 6

### Mã hóa nhị phân

In [50]:
# Định nghĩa các cột phân loại cho cả tập train và tập test
categorical_columns = ["Gender", "Ever_Married", "Graduated"]

In [51]:
# Sử dụng phương thức when để thực hiện mã hóa
df = df.withColumn("Gender", when(df["Gender"] == "Male", 0).otherwise(1))

In [52]:
from pyspark.sql.functions import when, col
# Sử dụng phương thức when để thực hiện mã hóa, giữ nguyên giá trị null
df = df.withColumn(
    "Ever_Married",
    when(col("Ever_Married") == "Yes", 1)
    .when(col("Ever_Married") == "No", 0)
    .otherwise(col("Ever_Married"))
)

In [53]:
from pyspark.sql.functions import when, col
# Sử dụng phương thức when để thực hiện mã hóa, giữ nguyên giá trị null
df = df.withColumn(
    "Graduated",
    when(col("Graduated") == "Yes", 1)
    .when(col("Graduated") == "No", 0)
    .otherwise(col("Graduated"))
)

In [54]:
# Tạo một danh sách chứa các tên cột và số lượng giá trị bị thiếu tương ứng
missing_values = []

# Lặp qua các cột trong DataFrame
for col in df.columns:
    # Tính tổng số giá trị bị thiếu trong cột và thêm vào danh sách
    missing_count = df.filter(df[col].isNull()).count()
    missing_values.append((col, missing_count))
# Hiển thị kết quả
print("Thống kê giá trị Null trong tập DataFrame:\n" + "-"*20)
for col, missing_count in missing_values:
    if missing_count != 0:
        print(f" - '{col}': {missing_count} giá trị Null.")


Thống kê giá trị Null trong tập DataFrame:
--------------------
 - 'Ever_Married': 185 giá trị Null.
 - 'Graduated': 99 giá trị Null.
 - 'Work_Experience': 1068 giá trị Null.
 - 'Family_Size': 442 giá trị Null.


In [55]:
df.show()

+------+------+------------+---+---------+----------+---------------+--------------+-----------+
|    ID|Gender|Ever_Married|Age|Graduated|Profession|Work_Experience|Spending_Score|Family_Size|
+------+------+------------+---+---------+----------+---------------+--------------+-----------+
|462809|     0|           0| 22|        0|         2|            1.0|             0|        4.0|
|460666|     1|           1| 49|        1|         9|           14.0|             0|        1.0|
|463622|     0|           0| 27|        1|         2|            8.0|             0|        1.0|
|467826|     0|           0| 18|        0|         2|            0.0|             0|        6.0|
|462040|     0|           0| 19|        0|         2|            5.0|             0|        4.0|
|467926|     0|           1| 79|        1|         1|            0.0|             2|        2.0|
|467761|     0|           0| 39|        1|         1|            1.0|             0|        2.0|
|459076|     0|           1| 6

## Chuẩn hóa một số cột

Các cột được chuẩn hóa theo StandardScaler: "Age", "Work_Experience", "Family_Size".

In [56]:
# Các cột cần chuẩn hóa
columns_to_scale = ['Age', 'Work_Experience', 'Family_Size']

In [57]:
# Áp dụng hàm standard_scaler
df = standard_scaler(df, columns_to_scale)

In [58]:
# Hiển thị kết quả
df.show()

+------+------+------------+--------------------+---------+----------+--------------------+--------------+-------------------+
|    ID|Gender|Ever_Married|                 Age|Graduated|Profession|     Work_Experience|Spending_Score|        Family_Size|
+------+------+------------+--------------------+---------+----------+--------------------+--------------+-------------------+
|462809|     0|           0| -1.2837627107909002|        0|         2| -0.4790680244349246|             0| 0.7505623307375167|
|460666|     1|           1| 0.32664871242027155|        1|         9|  3.3512099731580194|             0|-1.1996403317241193|
|463622|     0|           0| -0.9855383731592018|        1|         2|  1.5833893588843526|             0|-1.1996403317241193|
|467826|     0|           0|  -1.522342180896259|        0|         2| -0.7737047934805356|             0|  2.050697439045274|
|462040|     0|           0| -1.4626973133699193|        0|         2|  0.6994790517475196|             0| 0.75

## Xử lý giá trị Null

Các cột 'Age', 'Work_Experience', 'Family_Size' thì thay Null thành mean() tương ứng của nó bởi hàm calculate_mean()


In [59]:
numeric_columns = ['Age', 'Work_Experience', 'Family_Size'] # Nhóm đầu tiên gồm 3 cột: 'Age', 'Work_Experience', 'Family_Size' sẽ được thay thế các giá trị Null thành giá trị trung bình tương ứng của cột

for column in numeric_columns:
    mean_value = calculate_mean(df, column)  # Tính giá trị trung bình từng cột của df
    df = df.na.fill({column: mean_value}) # Điền những giá trị bị thiếu trong các cột trên thành giá trị trung bình

Các cột 'Gender', 'Ever_Married', 'Graduated', 'Profession', 'Segmentation' thay giá trị Null thành mode() của nó bởi hàm calculate_mode().

In [60]:
numeric_columns = ['Ever_Married', 'Graduated']
# Các cột mà điền Null thành giá trị xuất hiện nhiều nhất trong cột
for column in numeric_columns:
    mode_value = calculate_mode(df, column) # Tính giá trị xuất hiện nhiều nhất trong df của các cột trên
    df = df.na.fill({column: mode_value}) # Điền các giá trị bị thiếu trong tập thành giá trị xuất hiện nhiều nhất trong cột tương ứng

In [61]:
df.show()

+------+------+------------+--------------------+---------+----------+--------------------+--------------+-------------------+
|    ID|Gender|Ever_Married|                 Age|Graduated|Profession|     Work_Experience|Spending_Score|        Family_Size|
+------+------+------------+--------------------+---------+----------+--------------------+--------------+-------------------+
|462809|     0|           0| -1.2837627107909002|        0|         2| -0.4790680244349246|             0| 0.7505623307375167|
|460666|     1|           1| 0.32664871242027155|        1|         9|  3.3512099731580194|             0|-1.1996403317241193|
|463622|     0|           0| -0.9855383731592018|        1|         2|  1.5833893588843526|             0|-1.1996403317241193|
|467826|     0|           0|  -1.522342180896259|        0|         2| -0.7737047934805356|             0|  2.050697439045274|
|462040|     0|           0| -1.4626973133699193|        0|         2|  0.6994790517475196|             0| 0.75

In [62]:
# Tạo một danh sách chứa các tên cột và số lượng giá trị bị thiếu tương ứng
missing_values = []

# Lặp qua các cột trong DataFrame
for col in df.columns:
    # Tính tổng số giá trị bị thiếu trong cột và thêm vào danh sách
    missing_count = df.filter(df[col].isNull()).count()
    missing_values.append((col, missing_count))
# Hiển thị kết quả
print("Thống kê giá trị Null trong tập DataFrame:\n" + "-"*20)
for col, missing_count in missing_values:
    if missing_count != 0:
        print(f" - '{col}': {missing_count} giá trị Null.")

Thống kê giá trị Null trong tập DataFrame:
--------------------


**Nhận xét:** Không có cột nào được in ra vì không có cột nào chứa giá trị Null nữa.

In [63]:
import pandas as pd

# Chuyển đổi Spark DataFrame sang Pandas DataFrame
df= df.toPandas()

df.to_csv('customer_data.csv', index=False)